In [1]:
import pandas as pd
import os

In [8]:
# 엑셀 파일 경로
excel_path = "raw_data/cau_2025_timetable.xlsx"

# 엑셀 로드 (첫 시트 기준)
excel_df = pd.read_excel(excel_path, engine="openpyxl")

# DataFrame 확인
excel_df.head()

,대학,개설학과,학년,과정,이수구분,과목번호-분반,과목명,학점-시간,담당교수,폐강,강의시간,유연학기,비고,수업유형
0,대학(전체),NaN,3,학사,자유선택,54943-01,안전 및 조직관리 사례연구,3-6,학군단(R.O.T.C),NaN,"화0,1,2, 목0,1,2 / 305관 216호 <3학년 훈육관실>",NaN,R.O.T.C,NaN
1,대학(전체),NaN,3,학사,자유선택,54943-02,안전 및 조직관리 사례연구,3-6,학군단(R.O.T.C),NaN,"화10,11,12, 목10,11,12 / 305관 216호 <3학년 훈육관실>",NaN,R.O.T.C,NaN
2,대학(전체),NaN,4,학사,자유선택,54945-01,조직리더십 사례연구,3-6,학군단(R.O.T.C),NaN,"화0,1,2, 목0,1,2 / 305관 215호 <4학년 훈육관실>",NaN,R.O.T.C,NaN
3,대학(전체),NaN,4,학사,자유선택,54945-02,조직리더십 사례연구,3-6,학군단(R.O.T.C),NaN,"화10,11,12, 목10,11,12 / 305관 215호 <4학년 훈육관실>",NaN,R.O.T.C,NaN
4,대학(전체),NaN,전체,학사,교양,49950-01,ACT,2-2,최문정,NaN,"월1,2 / 310관 718호 <강의실>",NaN,공통교양 2학년 이상 수강 가능,NaN


In [9]:
# 1. 과목번호, 과목명, 담당교수, 폐강, 강의시간만 남기고 제거
columns_to_keep = ["과목번호-분반", "과목명", "담당교수", "폐강", "강의시간"]
excel_df = excel_df[columns_to_keep]
excel_df.head()

,과목번호-분반,과목명,담당교수,폐강,강의시간
0,54943-01,안전 및 조직관리 사례연구,학군단(R.O.T.C),NaN,"화0,1,2, 목0,1,2 / 305관 216호 <3학년 훈육관실>"
1,54943-02,안전 및 조직관리 사례연구,학군단(R.O.T.C),NaN,"화10,11,12, 목10,11,12 / 305관 216호 <3학년 훈육관실>"
2,54945-01,조직리더십 사례연구,학군단(R.O.T.C),NaN,"화0,1,2, 목0,1,2 / 305관 215호 <4학년 훈육관실>"
3,54945-02,조직리더십 사례연구,학군단(R.O.T.C),NaN,"화10,11,12, 목10,11,12 / 305관 215호 <4학년 훈육관실>"
4,49950-01,ACT,최문정,NaN,"월1,2 / 310관 718호 <강의실>"


In [10]:
# 1. 폐강 열이 NaN이 아닌 행 제거
excel_df = excel_df[excel_df["폐강"].isna()]

# 2. 폐강 열 제거
excel_df = excel_df.drop(columns=["폐강"])
excel_df.head()

,과목번호-분반,과목명,담당교수,강의시간
0,54943-01,안전 및 조직관리 사례연구,학군단(R.O.T.C),"화0,1,2, 목0,1,2 / 305관 216호 <3학년 훈육관실>"
1,54943-02,안전 및 조직관리 사례연구,학군단(R.O.T.C),"화10,11,12, 목10,11,12 / 305관 216호 <3학년 훈육관실>"
2,54945-01,조직리더십 사례연구,학군단(R.O.T.C),"화0,1,2, 목0,1,2 / 305관 215호 <4학년 훈육관실>"
3,54945-02,조직리더십 사례연구,학군단(R.O.T.C),"화10,11,12, 목10,11,12 / 305관 215호 <4학년 훈육관실>"
4,49950-01,ACT,최문정,"월1,2 / 310관 718호 <강의실>"


In [11]:
def convert_period_to_time(period_num: int) -> str:
    # 0교시: 8시 시작, 그 이후는 9시부터 시작
    base_hour = 8 if period_num == 0 else 9 + (period_num - 1)
    start = f"{base_hour:02d}:00"
    end = f"{base_hour + 1:02d}:00"
    return f"{start}~{end}"

import re

def parse_schedule(schedule_str: str):
    if pd.isna(schedule_str):
        return None, None, None

    # 시간 정보 추출

    # 예: '월10 / 310관 802호' 또는 '월(10:30~11:45) / 수10 / 303관 802호'
    parts = schedule_str.split('/')
    time_parts = []
    room_part = None

    for part in parts:
        part = part.strip()

        # 교시 형식 예: 월0, 수3 등
        match_period = re.match(r"([월화수목금토일])(\d+)", part)
        if match_period:
            day = match_period.group(1)
            period = int(match_period.group(2))
            time_range = convert_period_to_time(period)
            time_parts.append(f"{day}({time_range})")
            continue

        # 이미 시간으로 되어 있으면 그대로 추가
        match_time = re.match(r"([월화수목금토일])\(.+\)", part)
        if match_time:
            time_parts.append(part)
            continue

        # 강의실 정보로 간주
        if re.search(r"\d+관.*\d+호", part):
            room_part = part

    # 건물번호, 강의실 분리
    building = None
    classroom = None
    if room_part:
        match_room = re.match(r"(\d+관).*?(\d+-?\d*호)", room_part)
        if match_room:
            building = match_room.group(1)
            classroom = match_room.group(2)

    return " / ".join(time_parts), building, classroom

def split_time_info(time_str):
    if pd.isna(time_str):
        return None, None

    days = []
    times = []

    # 예: 월(10:30~11:45)
    parts = [p.strip() for p in time_str.split('/')]

    for part in parts:
        match = re.match(r"([월화수목금토일])\((\d{2}:\d{2}~\d{2}:\d{2})\)", part)
        if match:
            days.append(match.group(1))
            times.append(match.group(2))

    return days, times


# 강의시간, 건물번호, 강의실 파싱해서 새 컬럼으로 추가
excel_df[["강의시간_24시", "건물번호", "강의실"]] = excel_df["강의시간"].apply(
    lambda x: pd.Series(parse_schedule(x))
)

excel_df[["강의요일", "강의시간_리스트"]] = excel_df["강의시간_24시"].apply(
    lambda x: pd.Series(split_time_info(x))
)

excel_df = excel_df.drop(columns=["강의시간", "강의시간_24시"])

# 확인
excel_df.head()

,과목번호-분반,과목명,담당교수,건물번호,강의실,강의요일,강의시간_리스트
0,54943-01,안전 및 조직관리 사례연구,학군단(R.O.T.C),305관,216호,[화],[08:00~09:00]
1,54943-02,안전 및 조직관리 사례연구,학군단(R.O.T.C),305관,216호,[화],[18:00~19:00]
2,54945-01,조직리더십 사례연구,학군단(R.O.T.C),305관,215호,[화],[08:00~09:00]
3,54945-02,조직리더십 사례연구,학군단(R.O.T.C),305관,215호,[화],[18:00~19:00]
4,49950-01,ACT,최문정,310관,718호,[월],[09:00~10:00]


In [12]:
print(excel_df[excel_df["강의시간_리스트"].isnull()])


       과목번호-분반                  과목명     담당교수  건물번호   강의실  강의요일 강의시간_리스트
318   57104-01            공공기관NCS분석   인재개발센터  None  None  None     None
2373  58469-01               기초나노역학      유재형  None  None  None     None
2374  58583-01             나노소재물리화학      남인호  None  None  None     None
2375  58584-01               융합양자개론      이창연  None  None  None     None
2376  58582-01               나노기술입문      김영규  None  None  None     None
2377  58592-01                 나노센서  김승한 유재형  None  None  None     None
2378  58589-01            나노소재및박막역학      NaN  None  None  None     None
2379  58472-01               나노전자재료      김선주  None  None  None     None
2380  58590-01               나노포토닉스      김석민  None  None  None     None
2382  58585-01               전산나노과학      김영규  None  None  None     None
2386  58594-01            3D프린터활용입문      김석민  None  None  None     None
2387  58465-01              프로그래밍기초      김영규  None  None  None     None
2388  58596-01             적층제조구조역학      김영규  None  None  None  

In [13]:
print(excel_df[excel_df["강의요일"].isnull()])

       과목번호-분반                  과목명     담당교수  건물번호   강의실  강의요일 강의시간_리스트
318   57104-01            공공기관NCS분석   인재개발센터  None  None  None     None
2373  58469-01               기초나노역학      유재형  None  None  None     None
2374  58583-01             나노소재물리화학      남인호  None  None  None     None
2375  58584-01               융합양자개론      이창연  None  None  None     None
2376  58582-01               나노기술입문      김영규  None  None  None     None
2377  58592-01                 나노센서  김승한 유재형  None  None  None     None
2378  58589-01            나노소재및박막역학      NaN  None  None  None     None
2379  58472-01               나노전자재료      김선주  None  None  None     None
2380  58590-01               나노포토닉스      김석민  None  None  None     None
2382  58585-01               전산나노과학      김영규  None  None  None     None
2386  58594-01            3D프린터활용입문      김석민  None  None  None     None
2387  58465-01              프로그래밍기초      김영규  None  None  None     None
2388  58596-01             적층제조구조역학      김영규  None  None  None  

In [6]:
# zip 요일과 시간 리스트 → 튜플 목록
excel_df["temp"] = excel_df.apply(
    lambda row: list(zip(row["강의요일"], row["강의시간_리스트"])), axis=1
)

# explode
df_exploded = excel_df.explode("temp")

# 튜플만 남기기
df_exploded = df_exploded[df_exploded["temp"].apply(lambda x: isinstance(x, tuple) and len(x) == 2)]

# 튜플 분리
df_exploded[["요일", "시간"]] = pd.DataFrame(df_exploded["temp"].tolist(), index=df_exploded.index)
df_exploded = df_exploded.drop(columns=["temp", "강의요일", "강의시간_리스트"])

TypeError: 'NoneType' object is not iterable

In [77]:
df_exploded[["시작시간", "종료시간"]] = df_exploded["시간"].str.split("~", expand=True)
df_exploded = df_exploded.drop(columns=["시간"])

In [78]:
df_exploded = df_exploded.rename(columns={
    "건물번호": "building",
    "강의실": "room",
    "요일": "day",
    "시작시간": "start_time",
    "종료시간": "end_time",
    "과목번호-분반": "course_id",
    "과목명": "course_name",
    "담당교수": "professor"
})

In [79]:
df_exploded = df_exploded[
    ["building", "room", "day", "start_time", "end_time", "course_id", "course_name", "professor"]
]

In [80]:
# building에서 숫자만 추출 (예: "310관" → "310")
df_exploded["building"] = df_exploded["building"].apply(
    lambda x: re.search(r"\d+", str(x)).group() if pd.notna(x) else None
)

# room에서도 숫자만 추출 (예: "415호" 또는 "802-1호" → "415", "802-1")
df_exploded["room"] = df_exploded["room"].apply(
    lambda x: re.search(r"\d+(?:-\d+)?", str(x)).group() if pd.notna(x) else None
)

In [81]:
day_map = {
    "월": "monday",
    "화": "tuesday",
    "수": "wednesday",
    "목": "thursday",
    "금": "friday",
    "토": "saturday",
    "일": "sunday"
}

df_exploded["day"] = df_exploded["day"].map(day_map)

In [82]:
df_exploded = df_exploded.dropna(subset=["building", "room"])

In [83]:
df_exploded["building"] = df_exploded["building"].astype(int)


In [84]:
df_exploded.tail(20)

,building,room,day,start_time,end_time,course_id,course_name,professor
82,310,802,wednesday,10:30,11:45,56107-01,동태적 경제이론 (영어A강의),문지웅
83,310,921,friday,15:00,16:00,10235-02,산업조직론,김형진
86,310,612,monday,15:00,16:15,54461-02,응용계량경제학,NaN
86,310,612,wednesday,15:00,16:15,54461-02,응용계량경제학,NaN
87,310,927,monday,13:30,14:45,24658-01,화폐금융론,송원호
87,310,927,wednesday,13:30,14:45,24658-01,화폐금융론,송원호
88,310,311,tuesday,13:30,14:45,24658-02,화폐금융론,김태진
88,310,311,thursday,13:30,14:45,24658-02,화폐금융론,김태진
89,310,505,thursday,12:00,13:00,24658-03,화폐금융론 (영어A강의),구세범
90,310,921,monday,09:00,10:15,01452-01,경제정책론 (영어A강의),고선


In [85]:
json_data = df_exploded.to_dict(orient="records")

save_dir = "converted_data"
os.makedirs(save_dir, exist_ok=True)

import json

save_path = os.path.join(save_dir, "room_schedule.json")

with open(save_path, "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)

print(f"✅ JSON 저장 완료: {save_path}")


✅ JSON 저장 완료: converted_data\room_schedule.json
